<a href="https://colab.research.google.com/github/Fryingpannn/WallStreetBets_BigDataAnalysis/blob/main/ProjectWallStreetBets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install praw
!pip3 install iexfinance

     |████████████████████████████████| 212.3MB 61kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b24e4831a56cedb1ebbe7536a61ceb7a5068071b6daed26bdd4bc581593d9690
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 163kB 4.4MB/s 
     |████████████████████████████████| 61kB 3.8MB/s 


In [166]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
#from iexfinance.stocks import Stock
#from iexfinance.stocks import get_historical_data
import os
# tools
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import string
# Set IEX Finance API Token for Sandbox test mode
os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
os.environ['IEX_TOKEN'] = 'Tsk_4060833567884b49870980c4a917aa92'

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark
spark = init_spark()

In [167]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [168]:
"""
Function to make an HTTP request to the Pushshift API.
- max_retry: Nb of times the request is re-tried if failure occurs.
- returns: Python object with the content of the request. (index 'data' property)
"""
def get_request(uri, max_retry = 5):
  def get(uri):
    response = requests.get(uri)
    assert response.status_code == 200
    return json.loads(response.content)
  # Retry if request call failed
  retry = 1
  while retry < max_retry:
    try:
      response = get(uri)
      return response
    except:
      #print(f"[{retry}] Request failed, re-trying...")
      # wait 1 second before retry
      time.sleep(1)
      retry += 1

In [4]:
# Testing get_request() with test uri. Should return a non-empty Python object.
obj = get_request("https://httpbin.org/get")
print(obj['url'])
# Returning posts from wallstreetbets. The posts are in the "data" property of the response.
# Use this to check format: https://jsonformatter.curiousconcept.com/#
# obj2 = get_request('https://api.pushshift.io/reddit/search/submission?subreddit=wallstreetbets')
# print(obj2)

https://httpbin.org/get


In [195]:
"""
Gets the all the posts from a given subreddit in the specific time range.
- subreddit: name of subreddit
- begin: timestamp (in unix) of start date
- end: timestamp (in unix) of end date
- returns: list of all the posts in the time interval. the posts are objects with properties "id", "title" and "creation_utc".
"""
def get_posts(subreddit, begin, end):
  # Max size of each Pushshift API request is 100 posts.
  SIZE = 100
  PUSHSHIFT_URI = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
  nb_requests_made = 1
  loading_string = "LOADING %: ["
  print("LOADING %: [", end=" ")
  # Get the ids and creation time of the posts only. Can use later to get the actual posts with PRAW with these ids.
  # Alternatively, we could also directly use get_request() instead of this function, and get all the posts with their content.
  def filter_ids_time(uri, begin, end):
    full_posts = get_request(uri.format(subreddit, begin, end, SIZE))
    if full_posts is None:
      raise ValueError("Response is empty or none.")

    # only keep the id, and creation time.
    """
    filtered = map(lambda post: {
        'id': post['id'],
        'created_utc': post['created_utc'],
        'flair': post['link_flair_text'],
        'flaircss': post['link_flair_css_class'],
        'text': post['selftext']
    }, full_posts['data'])
    """
    posts = []
    for post in full_posts['data']:
      try:
        if post['link_flair_text'] and post['link_flair_css_class'] and post['selftext'] and post['selftext'] != "[removed]" and post['selftext'] != "[deleted]":
          posts.append({\
            'id': post['id'],\
            'title': post['title'].translate(str.maketrans('', '', string.punctuation)),\
            'score': post['score'],\
            'upvote_ratio': post['upvote_ratio'],\
            'author': post['author'],\
            'created_utc': post['created_utc'],\
            'flair': post['link_flair_text'],\
            'flaircss': post['link_flair_css_class'],\
            'num_comments': post['num_comments'],\
            'text': post['selftext'].translate(str.maketrans('', '', string.punctuation)),\
            'url': post['url']\
          })
      except:
        pass
    # get timestamp of last post
    last_timestamp = full_posts['data'][-1]['created_utc']
    posts_amount = len(full_posts['data'])
          
    #return list(filtered)
    return [posts, last_timestamp, posts_amount]

  posts_etc = filter_ids_time(PUSHSHIFT_URI, begin, end)
  posts = posts_etc[0]
  last_timestamp = posts_etc[1]
  posts_amount = posts_etc[2]
  # If reached limit of 100 posts retrieved, make request again until 'end' time.
  while posts_amount == SIZE:
    # Timestamp of the last post we previously retrieved
    #new_begin = posts[-1]['created_utc'] - 10
    new_begin = last_timestamp - 10
    more_posts_etc = filter_ids_time(PUSHSHIFT_URI, new_begin, end)
    last_timestamp = more_posts_etc[1]
    posts_amount = more_posts_etc[2]
    posts.extend(more_posts_etc[0])
    nb_requests_made += 1
    printProgressBar((last_timestamp-begin)//10000 + 1, (end-begin)//10000)
    
  print("Number of requests made: ", nb_requests_made)
  return posts

In [162]:
"""
============================== USE THE OTHER FUNCTION BELOW ===========================

Testing get_posts() function.
- Timestamp converter: https://www.unixtimestamp.com/index.php?ref=theredish.com%2Fweb
- Able to retrieve up till latest posts from 6 hours ago.
- To print till "now": math.ceil(datetime.utcnow().timestamp())
"""
# Posts from March 13th
#posts = get_posts('wallstreetbets', 1615687200, math.ceil(datetime.utcnow().timestamp()))
# All posts from nb_days_from_today
nb_days_from_today = 90
end = math.ceil(datetime.utcnow().timestamp())
begin = math.ceil((datetime.utcnow() - timedelta(days=nb_days_from_today)).timestamp())
print("Timestamps: ", begin, end)
posts = get_posts('wallstreetbets', 1615824107, 1615838507)
unique_posts = np.unique([post['id'] for post in posts])
# Use np.unique to get rid of duplicates and filter posts only by id (only need id for praw).
print("Size: ", len(posts))
print("Size of uniques: ", len(unique_posts))
print("Example posts: ", posts[:5])

Timestamps:  1609210077 1616986077
[1] Request failed, re-trying...
[1] Request failed, re-trying...
[1] Request failed, re-trying...
Number of requests made:  13
Size:  363
Size of uniques:  356
Example posts:  [{'id': 'm5nyck', 'title': 'AMC is rising', 'score': 1, 'upvote_ratio': 1.0, 'author': 'Sufficient-Ad4354', 'created_utc': 1615824137, 'flair': 'Gain', 'flaircss': 'profit', 'num_comments': 2, 'text': 'AMC will hit 15 today at this rate', 'url': 'https://www.reddit.com/r/wallstreetbets/comments/m5nyck/amc_is_rising/'}, {'id': 'm5nyja', 'title': 'AMC to the moon', 'score': 2, 'upvote_ratio': 1.0, 'author': 'Ariasnyc', 'created_utc': 1615824150, 'flair': 'Gain', 'flaircss': 'profit', 'num_comments': 2, 'text': '🌚🌚🌚', 'url': 'https://www.reddit.com/r/wallstreetbets/comments/m5nyja/amc_to_the_moon/'}, {'id': 'm5nyq4', 'title': 'GMEs rise was predicted by Pink Floyd an extensive study', 'score': 1, 'upvote_ratio': 1.0, 'author': 'Ichthasen', 'created_utc': 1615824163, 'flair': 'Meme

In [196]:
"""
Retrieve posts
- nb_days_from_today: number of days from today you want to retrieve
- return: lists of all posts in time interval
"""

def retrieve(nb_days_from_today):
  end = math.ceil(datetime.utcnow().timestamp())
  begin = math.ceil((datetime.utcnow() - timedelta(days=nb_days_from_today)).timestamp())
  print("Timestamps: ", begin, end)
  posts = get_posts('wallstreetbets', begin, end)

  unique_posts = np.unique([post['id'] for post in posts])
  print("Size: ", len(posts))
  print("Size of uniques: ", len(unique_posts))
  print("Example posts: ", posts[:5])
  return posts

posts = retrieve(1)

Timestamps:  1616900737 1616987137
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of requests made:  13
Size:  161
Size of uniques:  159
Example posts:  [{'id': 'mete3k', 'title': 'CLOV', 'score': 1, 'upvote_ratio': 1.0, 'author': 'm4rcolio', 'created_utc': 1616901443, 'flair': 'Discussion', 'flaircss': 'question', 'num_comments': 0, 'text': 'Anyone here on CLOV Trying to decide if I should buy more since it’s at 7 Looks like Hindenburg and their partners are shorting it pretty bad until April when the claims expire Thoughts', 'url': 'https://www.reddit.com/r/wallstreetbets/comments/mete3k/clov/'}, {'id': 'methj8', 'title': 'Is 50K a short seller or algo', 'score': 1, 'upvote_ratio': 1.0, 'author': 'sDesiderio', 'created_utc': 1616901794, 'flair': 'Technical Analysis', 'flaircss': 'chart', 'num_comments':

In [161]:
"""
OUTPUT TO CSVVVVVVV
"""
posts_rdd = spark.sparkContext.parallelize(posts)
posts_rdd = posts_rdd.map(lambda post: Row(id=post['id'], title=post['title'], flair=str(post['flair']), score=post['score'], upvote_ratio=post['upvote_ratio'], author=str(post['author']), num_comments=post['num_comments'], text=post['text'], created=post['created_utc'], url=post['url']))
posts_rdd = posts_rdd.distinct()
posts_df = spark.createDataFrame(posts_rdd)
print(posts_df.count())
posts_df.show()

posts_df.coalesce(1).write.csv('30daysredditposts.csv', header=True)

15397
+------+--------------------+----------+-----+------------+-------------------+------------+--------------------+----------+--------------------+
|    id|               title|     flair|score|upvote_ratio|             author|num_comments|                text|   created|                 url|
+------+--------------------+----------+-----+------------+-------------------+------------+--------------------+----------+--------------------+
|ltep3k|I shopped at Game...|Discussion| 7057|        0.96|             nrouns|         821|I went into a Gam...|1614394911|https://www.reddi...|
|ltfinj|What would you do...|Discussion|   35|        0.79|         fzctungkun|         148|Saw news flash th...|1614397662|https://www.reddi...|
|ltfz6j|For those of you ...|      Meme|  182|        0.96|         Gyrene4341|          23|httpspreviewreddi...|1614399235|https://www.reddi...|
|ltg3fi|GME Creyon graph ...|     Chart|   31|        0.78|     StephenElliott|          39|Im just a retard ...|16143

In [65]:
import praw
import os
"""
Connect to Reddit API using environment variables. (values from reddit app)
- returns: reddit instance
"""
def connect_reddit():
  reddit = praw.Reddit(
    username = "fryingpannnnnn", #os.environ['REDDIT_NAME'],
    password = "wsbpan123", #os.environ['REDDIT_PASS'],
    client_secret = "GWFFdjptuoOsDPjpxERQsBY-xV7GfQ", #os.environ['REDDIT_SECRET'],
    client_id = "VWDfl0X3JFVwWQ", #os.environ['REDDIT_CLIENTWSB'],
    user_agent = "wsbscrape",
    check_for_async=False
  )
  return reddit

In [66]:
# Testing connect_reddit() function
# Should print "<praw.reddit.Reddit object at ...>"
reddit = connect_reddit()
print(reddit)

In [67]:
"""
Function to retrieve all Reddit posts of given ids.
- unique_posts: the id of the posts you want to retrieve.
- returns: list of reddit submission instances (reddit posts) of the input ids.
"""
def get_reddit_posts(unique_posts):
  reddit = connect_reddit()
  PAUSE_BEFORE_NEXT_CALL = 0.3

  reddit_posts = []

  for index,post_id in enumerate(unique_posts):
    submission = reddit.submission(id=post_id)
    if submission.link_flair_text and submission.selftext != "[removed]" and submission.selftext != "[deleted]" and submission.selftext:
      reddit_posts.append(submission)
      print(index, submission.title)
    time.sleep(PAUSE_BEFORE_NEXT_CALL)

  return reddit_posts

In [68]:
# Testing get_reddit_posts(): retrieving 1000 posts.
reddit_posts = get_reddit_posts(posts)
print("Size of posts: ", len(reddit_posts))
print("First 5 posts: ", reddit_posts[:10])

58 $FSR Updated Price Target: $4.21
95 My DD on the the perfect storm for Americas largest steel manufacture....CLF
398 Tiger Cub Archegos Liquidation and Friday’s market action in some of the stocks affected
403 Move over UWMC, PFSI is the most undervalued large mortgage company
535 [$BB] The size of real-world driving datasets that Blackberry IVY has access to will be so much bigger than Tesla's
895 Biden's Infrastructure Plan and BLNK moon potential?
923 Blackberry QNX + IVY might be the next Android + Google Mobile Services
933 Battle of the Fallen Giants: ViacomCBS vs DISCOVERY COMMUNICATIONS INC. (A shallow comparison of them both.)
968 Why Cannabis Legalization is BEARISH for some MSOs and BULLISH AF for LPs like $APHA/$TLRY 🚀🚀🚀
972 QNX hypervisor will be in all future connected EVs (possibly including Tesla as self-driving and vehicle software becomes more regulated)
1084 $Root why I like this stock.
1134 ASO - What Do You Think?
1167 Cathie Wood's ARK purchased more than 1 mil

In [112]:
reddit_posts[8].selftext = reddit_posts[8].selftext.translate(str.maketrans('', '', string.punctuation))
reddit_posts[8].selftext

'This weekend New York legislators finally agreed on all details of their marijuana legalization bill and expect it to be voted on in the coming days It is GREAT news for LPs and pretty worrying for retail dependant MSOs like Trulieve or Curaleaf but great news for wholesale based MSOs like Cresco or 4 Front Article about NY weed from our favorite shills CNBChttpswwwgooglecomampswwwnbcnewscomnewsampncna1262266 The interesting part of NYs bill is that they are implementing a TIERED system Meaning MSOs will NOT be allowed to create an oligarchy like they are accustomed to and rely upon Some companies will be grandfathered in but even those grandfathered will come with some pretty shitty limitations on expansion LPs entering the market later will be in a better position than grandfathered MSOs by multiples It is also interesting that New York believes that their model will be the ideal national model That would have some massive consequences for MSOs When this bill becomes law New York wi

In [115]:
posts_rdd = spark.sparkContext.parallelize(reddit_posts)
posts_rdd = posts_rdd.map(lambda post: Row(title=post.title, flair=post.link_flair_text, score=post.score, upvote_ratio=post.upvote_ratio, author=str(post.author), text=post.selftext.translate(str.maketrans('', '', string.punctuation)), created=post.created, url=post.url))
posts_df = spark.createDataFrame(posts_rdd)
posts_df.show()

posts_df.coalesce(1).write.csv('8testredditposts.csv', header=True)

+--------------------+----------------+-----+------------+--------------------+--------------------+-------------+--------------------+
|               title|           flair|score|upvote_ratio|              author|                text|      created|                 url|
+--------------------+----------------+-----+------------+--------------------+--------------------+-------------+--------------------+
|$FSR Updated Pric...|              DD|   60|        0.73|             fib_seq|So I wrote up a t...|1.616904131E9|https://www.reddi...|
|My DD on the the ...|              DD|  286|        0.86|          Matacumbie|If you like tendi...|1.616906325E9|https://www.reddi...|
|Tiger Cub Archego...|            News|  342|        0.96|           yintrepid|Several news outl...|1.616921735E9|https://www.reddi...|
|Move over UWMC, P...|              DD|    0|        0.35|         CMScientist|First a summary

...|1.616922003E9|https://www.reddi...|
|[$BB] The size of...|              DD|  960|   

In [56]:
"""
Filter posts
"""
def filter_posts(reddit_posts):
  posts_rdd = spark.sparkContext.parallelize(reddit_posts)
  posts_rdd = posts_rdd.filter(lambda post: post.selftext != "[removed]" and post.selftext != "[deleted]" and post.selftext)
  #posts_rdd = posts_rdd.map(lambda post: Row(ticker=get_ticker(post.title), title=post.title, flair=post.link_flair_text, score=post.score, upvote_ratio=post.upvote_ratio, author=str(post.author), text=post.selftext, created=post.created, url=post.url))
  #posts_df = spark.createDataFrame(posts_rdd)
  return posts_rdd


In [57]:
# test filter_posts
all_posts = filter_posts(reddit_posts)
all_posts.take(20)

KeyboardInterrupt: ignored

In [37]:
posts_rdd = spark.sparkContext.parallelize(reddit_posts)
posts_rdd = posts_rdd.filter(lambda post: post.selftext != "[removed]" and post.selftext != "[deleted]" and post.selftext)
posts_rdd.collect()

KeyboardInterrupt: ignored

In [29]:
posts_rdd = posts_rdd.map(lambda post: Row(ticker=get_ticker(post.title), title=post.title, flair=post.link_flair_text, score=post.score, upvote_ratio=post.upvote_ratio, author=str(post.author), text=post.selftext, created=post.created, url=post.url))
posts_df = spark.createDataFrame(posts_rdd)
posts_df.show()

+--------------------+-----+------------+-------------------+---------+-------------+--------------------+
|               title|score|upvote_ratio|             author|     text|      created|                 url|
+--------------------+-----+------------+-------------------+---------+-------------+--------------------+
|Finally recovered...|   52|        0.87|      WarriorAlways|[removed]|1.616901491E9|https://www.reddi...|
|Enter the Roaring...|    1|         1.0|               None|[deleted]|1.616901505E9|https://v.redd.it...|
|New to investing,...|    1|         1.0|      eevee-lutions|[removed]|1.616901508E9|https://www.reddi...|
|💎GME💎Price in L...|    4|        0.75|     Money-Maker111|         |1.616901552E9|https://v.redd.it...|
|I, too, took this...|  114|         0.9|         l337joejoe|         |1.616901569E9|https://i.redd.it...|
|Enter the Roaring...|   21|        0.92|Inevitable-Elk-4162|         | 1.61690162E9|https://v.redd.it...|
|My Beef Lol Mein ...|    1|         1.

In [33]:
filter_posts = posts_df.na.drop()
filter_posts = filter_posts.filter(filter_posts.text != "[removed]" && filter_posts.text != "[deleted]" && filter_posts.text)
filter_posts.take(4)

KeyboardInterrupt: ignored

In [32]:
posts_df.coalesce(1).write.csv('redditposts.csv')

KeyboardInterrupt: ignored